In [150]:
import pandas as pd
import numpy as np
import json
from pathlib import Path  
from ast import literal_eval

In [7]:
pd.set_option('display.max_rows', None)

In [103]:
with open("tools/state_codes.json", 'r') as f:
    state_codes = json.load(f)

with open("tools/countyfy.json", 'r') as f:
    county_dict_raw = json.load(f)

county_dict = {}
for key in county_dict_raw:
    county_dict[literal_eval(key.replace('~', '\\\''))] = county_dict_raw[key]

In [146]:
col_data = "datasets/cost-of-living-database.xlsx"
MIN_COUNTY_SIZE = 90000

In [21]:
col_food = pd.read_excel(col_data, sheet_name="Food Costs")
col_rent = pd.read_excel(col_data, sheet_name="Housing Costs - Rent")

col_rent_temp = col_rent
col_rent_temp.columns = col_rent_temp.iloc[1]
col_rent_temp = col_rent_temp.drop([0,1])
col_rent_temp = col_rent_temp[(col_rent_temp["numadults"] == 1) & 
                              (col_rent_temp["numkids"] == 0) & 
                              ((col_rent_temp["countyortownpop"] >= MIN_COUNTY_SIZE) |
                               (col_rent_temp["stateAbbrev"].isin(["CT", "DC", "ME", "NH", "RI", "VA"])))]
col_rent_temp = col_rent_temp[["stateAbbrev", "countyortownName", "expense.rent", "countyortownpop"]]
# col_rent_temp[]

In [22]:
display(col_rent_temp)

1,stateAbbrev,countyortownName,expense.rent,countyortownpop
17,AL,Baldwin County,8069.714278,182265
107,AL,Calhoun County,4949.714278,118572
407,AL,Etowah County,4793.714278,104430
512,AL,Houston County,5021.714278,101547
542,AL,Jefferson County,7949.714278,658466
602,AL,Lee County,6317.714278,140247
662,AL,Madison County,7013.714278,334811
722,AL,Mobile County,6965.714278,412992
752,AL,Montgomery County,7145.714278,229363
767,AL,Morgan County,5129.714278,119490


In [131]:
col_rent_temp_test = col_rent_temp

In [132]:
countied = {}

In [134]:
def combineRows(row):
    name = " ".join(row[1].split()[:-1])
    if (name, row[0]) not in county_dict:
#         print((name, row[0]))
        return
    
    if county_dict[(name, row[0])] not in countied:
        countied[county_dict[(name, row[0])]] = {"stateAbbrev": [row[0]], 
                                                 "countyortownName": [county_dict[(name, row[0])]], 
                                                 "expense.rent": [row[2]], 
                                                 "countyortownpop": [0]}
        
    temp = countied[county_dict[(name, row[0])]]
    countied[county_dict[(name, row[0])]]["countyortownpop"][0] += row[3]
    return countied

col_rent_temp_test.apply(combineRows, axis=1)
# col_rent_temp_test

17                                                    None
107                                                   None
407                                                   None
512                                                   None
542                                                   None
602                                                   None
662                                                   None
722                                                   None
752                                                   None
767                                                   None
872                                                   None
932                                                   None
1037                                                  None
1457                                                  None
1472                                                  None
1547                                                  None
1562                                                  No

In [135]:
countied

{'Fairfield County': {'stateAbbrev': ['CT'],
  'countyortownName': ['Fairfield County'],
  'expense.rent': [13829.7142781554],
  'countyortownpop': [916829]},
 'Hartford County': {'stateAbbrev': ['CT'],
  'countyortownName': ['Hartford County'],
  'expense.rent': [10313.7142781554],
  'countyortownpop': [894014]},
 'Litchfield County': {'stateAbbrev': ['CT'],
  'countyortownName': ['Litchfield County'],
  'expense.rent': [9485.71427815536],
  'countyortownpop': [189927]},
 'Middlesex County': {'stateAbbrev': ['CT'],
  'countyortownName': ['Middlesex County'],
  'expense.rent': [10313.7142781554],
  'countyortownpop': [377357]},
 'New Haven County': {'stateAbbrev': ['CT'],
  'countyortownName': ['New Haven County'],
  'expense.rent': [12125.7142781554],
  'countyortownpop': [862477]},
 'New London County': {'stateAbbrev': ['CT'],
  'countyortownName': ['New London County'],
  'expense.rent': [9569.71427815536],
  'countyortownpop': [274055]},
 'Tolland County': {'stateAbbrev': ['CT'],
 

In [136]:
for new_county in countied:
    col_rent_temp = pd.concat([col_rent_temp, pd.DataFrame(countied[new_county])], ignore_index=True)

In [157]:
col_rent = col_rent_temp.loc[col_rent_temp["countyortownpop"] >= MIN_COUNTY_SIZE].sort_values(['stateAbbrev', 'countyortownName']).reset_index().drop(columns=["index"])
col_rent = col_rent.rename(columns={"stateAbbrev": "state", "countyortownName": "county", "countyortownpop": "pop"})

In [148]:
len(col_rent)

587

In [158]:
col_rent

,state,county,expense.rent,pop
0,AK,Anchorage Municipality,10853.714278,291826
1,AL,Baldwin County,8069.714278,182265
2,AL,Calhoun County,4949.714278,118572
3,AL,Etowah County,4793.714278,104430
4,AL,Houston County,5021.714278,101547
5,AL,Jefferson County,7949.714278,658466
6,AL,Lee County,6317.714278,140247
7,AL,Madison County,7013.714278,334811
8,AL,Mobile County,6965.714278,412992
9,AL,Montgomery County,7145.714278,229363


In [159]:
col_rent.to_csv("./dataframes/col_rent")